In [1]:
# Step 1: Import the packages
try:
    import onnx
    from onnx import helper, TensorProto
    import onnxruntime as ort
    import numpy as np
    import os
    print("Step 1: Importing onnx, onnxruntime, numpy, and os passed.")
except ImportError:
    print("Step 1 failed: 'onnx', 'onnxruntime', 'numpy', or 'os' is not installed.")
    sys.exit(1)
except Exception as e:
    print(f"Step 1 failed with an unexpected error: {str(e)}")
    sys.exit(1)

# Step 2: Create a simple ONNX model
try:
    # Define the model input (1x3x224x224 tensor)
    input_tensor = helper.make_tensor_value_info('input', TensorProto.FLOAT, [1, 3, 224, 224])

    # Define the model output (1x3x224x224 tensor)
    output_tensor = helper.make_tensor_value_info('output', TensorProto.FLOAT, [1, 3, 224, 224])

    # Create a simple node that performs an element-wise addition of the input with itself
    add_node = helper.make_node(
        'Add',  # Operator name
        ['input', 'input'],  # Inputs
        ['output']  # Outputs
    )

    # Create the graph
    graph_def = helper.make_graph(
        [add_node],
        'simple-addition-graph',  # Graph name
        [input_tensor],  # Inputs
        [output_tensor]  # Outputs
    )

    # Create the model, specifying opset version 21
    model_def = helper.make_model(graph_def, producer_name='onnx-example', opset_imports=[helper.make_opsetid("", 21)])
    print("Step 2: Creating a simple ONNX model passed.")
except Exception as e:
    print(f"Step 2 failed: {str(e)}")
    sys.exit(1)

# Step 3: Validate the created ONNX model
try:
    onnx.checker.check_model(model_def)
    print("Step 3: Model validation passed.")
except onnx.checker.ValidationError as e:
    print(f"Step 3 failed: Model validation error - {str(e)}")
    sys.exit(1)
except Exception as e:
    print(f"Step 3 failed: {str(e)}")
    sys.exit(1)

# Step 4: Save the ONNX model to a file
try:
    model_path = 'simple_model.onnx'
    onnx.save(model_def, model_path)
    assert os.path.exists(model_path), "Model save failed, file not created."
    print(f"Step 4: Saving the ONNX model to '{model_path}' passed.")
except Exception as e:
    print(f"Step 4 failed: {str(e)}")
    sys.exit(1)

# Step 5: Perform inference using ONNX Runtime
try:
    # Create an ONNX Runtime session
    session = ort.InferenceSession(model_path)
    
    # Generate a dummy input matching the model's input shape
    dummy_input = np.random.randn(1, 3, 224, 224).astype(np.float32)
    
    # Perform inference
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    result = session.run([output_name], {input_name: dummy_input})
    
    assert result is not None, "Inference failed, no output received."
    assert np.allclose(result, dummy_input + dummy_input), "Inference result is not as expected."
    print("Step 5: Performing inference using ONNX Runtime passed.")
except Exception as e:
    print(f"Step 5 failed: {str(e)}")
    sys.exit(1)

# Step 6: Explore ONNX Runtime session details
try:
    # Check input and output details
    input_details = session.get_inputs()[0]
    output_details = session.get_outputs()[0]
    
    assert input_details.shape == [1, 3, 224, 224], f"Unexpected input shape: {input_details.shape}"
    assert output_details.shape == [1, 3, 224, 224], f"Unexpected output shape: {output_details.shape}"
    
    print("Step 6: Exploring ONNX Runtime session details passed.")
    print(f"Input details: {input_details}")
    print(f"Output details: {output_details}")
except Exception as e:
    print(f"Step 6 failed: {str(e)}")
    sys.exit(1)

# Final Confirmation
print("All extensive tests for the 'onnxruntime' package completed successfully.")


Step 1: Importing onnx, onnxruntime, numpy, and os passed.
Step 2: Creating a simple ONNX model passed.
Step 3: Model validation passed.
Step 4: Saving the ONNX model to 'simple_model.onnx' passed.
Step 5: Performing inference using ONNX Runtime passed.
Step 6: Exploring ONNX Runtime session details passed.
Input details: NodeArg(name='input', type='tensor(float)', shape=[1, 3, 224, 224])
Output details: NodeArg(name='output', type='tensor(float)', shape=[1, 3, 224, 224])
All extensive tests for the 'onnxruntime' package completed successfully.
